In [1]:
# Definición de la funcion que recupera la información meteorológica de las estaciones de observación 
# existentes entre dos periodos de tiempo datos de la BBDD de Temperaturas Mundiales

# El resultado aparecerá agrupado por continente

import sqlite3
import pandas as pd
from pandas import DataFrame, Series
import time

def ejecucion_continentes(bbdd_file,continents,init_year,ending_year):
    # Abrimos la conexión con la BBDD
    conn = sqlite3.connect(bbdd_file)
    cursor = conn.cursor()

    # Vamos a intentar automatizar el análisis anterior, es decir, considerar los datos a partir de cierto año y a 
    # continuacion filtrar por aquellas estaciones que tienen observaciones por año inferior a 364 dias
    df_total = DataFrame()
    df_estaciones = DataFrame()

    for cont in continents:
        print("Inicio continente: " + cont + " a las " + str(time.strftime("%H:%M:%S")))
        
        # Se recuperan todas las estaciones del continentes que existían previamente a 1900 y existen en 2016
        rows = cursor.execute("""select distinct station 
                                 from world_time_"""+cont+"""
                                 where date = '"""+str(init_year)+"0101';")

        station_list_init = []
        for r in rows:
            station_list_init.append(r[0])
            
        print ("   Numero estaciones en el año inicial: " + str(len(station_list_init)))

        if len(station_list_init) > 0:
            rows = cursor.execute("""select distinct station 
                                     from world_time_"""+cont+"""
                                     where date = '"""+str(ending_year)+"1231';")        
            station_list_ending = []
            for r in rows:
                station_list_ending.append({"station":r[0]})
                
            print ("   Numero estaciones en el año final: " + str(len(station_list_ending)))
                        
            station_list_ending = DataFrame(station_list_ending)
            station_list_ending = DataFrame(station_list_ending[station_list_ending.station.map(lambda x: x in station_list_init)])
            station_list_ending = station_list_ending.reset_index()[["station"]]
            print ("   Numero estaciones comunes: " + str(len(station_list_ending)))

            # Se itera de 100 en 100 estaciones
            df = DataFrame()
            station_list_ending = station_list_ending.station
            while(len(station_list_ending) > 0):
                if len(station_list_ending) > 100:
                    new = station_list_ending[:100]
                    station_list_ending = station_list_ending[100:]
                else:
                    new = station_list_ending
                    station_list_ending = []
                
                stats = ""
                for s in new:
                    if stats != "":
                        stats = stats + ","
                    stats = stats + "'" + s + "'" 
                                
                # De todas las estaciones anteriores, se recupera el número de observaciones no nulas por año
                rows = cursor.execute("""select date, substr(date,1,4) as year, station, tmax, tmin, 1 as n_records
                                         from world_time_"""+cont+"""
                                         where station in (""" + stats + """) and 
                                         date >= '""" + str(init_year) + """0101';""")

                data = DataFrame()
                result = rows.fetchmany(400000)
                while len(result) > 0:
                    data = data.append(DataFrame(result))
                    result = rows.fetchmany(400000)                
                    
                print("   Numero de registros recuperados de las estaciones: " + str(len(data)))
                
                if len(data) > 0:
                    data.columns = ["date","year","station","tmax","tmin","n_records"]                
                    data = data.dropna()
                    data = data[data.tmin > -60.0]
                    data = data[data.tmax < 60.0]
                    data = data[data.year.map(lambda x: int(x) >= init_year)]
                    data = data[data.year.map(lambda x: int(x) <= ending_year)]
                    data_year = data[["year","station","n_records"]].groupby(["year","station"]).sum()
                    data_year = data_year.reset_index()
                                
                    # Nos quedamos con aquellas estaciones que tienen menos de 360 observaciones por año
                    less_360_obs = data_year[data_year.n_records < 360][["station"]].drop_duplicates().station
                                
                    # Eliminamos las estaciones con menos de 360 observaciones del dataframe de datos
                    if len(less_360_obs) > 0:
                        data = data[data.station.map(lambda x: x not in less_360_obs.as_matrix())]
                        data_year = data_year[data_year.station.map(lambda x: x not in less_360_obs.as_matrix())]
                    
                    print("   Numero de estaciones: " + str(len(data.station.drop_duplicates())))
                
                    less_360_obs = []
                    with_gaps = []
                    
                    # Se buscan las estaciones que tienen al menos un año sin información
                    for s in data_year[["station"]].drop_duplicates().station:
                        sub_data_year = data_year[data_year.station == s]
                        prev_year = init_year-1
                        ind_gap = 0
                        for y in sub_data_year[["year"]].year:
                            if int(y) - prev_year > 1 and ind_gap == 0:
                                ind_gap = ind_gap + 1
                                with_gaps.append(s)
                                
                            prev_year = int(y)

                        if ending_year - int(y) != 0 and ind_gap == 0:
                            with_gaps.append(s)
                        
                    # Se eliminan todas las estaciones con gaps
                    if len(with_gaps) > 0:
                        data = data[data.station.map(lambda x: x not in with_gaps)]
                
                    print("   Numero de estaciones: " + str(len(data.station.drop_duplicates())))
                
                    with_gaps = []
                    sub_data_year = DataFrame()
                    data_year = DataFrame()                
                    df = df.append(data)
                    sub_data = DataFrame()
                    
    conn.close()


In [4]:
import numpy as np
import matplotlib.pyplot as plt

t = np.arange(0.01, 10.0, 0.01)
s1 = np.exp(t)
s2 = np.sin(2*np.pi*t)


fig, ax1 = plt.subplots()
ax1.plot(t, s1, 'b-')
ax2 = ax1.twinx()
ax2.plot(t, s2, 'r.')


#ax2.set_ylabel('sin', color='r')

plt.show()
#ax1.set_xlabel('time (s)')
# Make the y-axis label and tick labels match the line color.
#ax1.set_ylabel('exp', color='b')
    